<a href="https://colab.research.google.com/github/AUT-Student/NN-HW4/blob/main/NN_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: Neural Network
<br>
<b>Description:</b> HomeWork 4
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds

from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Flatten, Dropout
from keras.layers import MaxPooling2D, AveragePooling2D, GlobalMaxPooling2D
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2
from keras.applications.inception_v3 import InceptionV3
from keras.models import clone_model

# Dataset

In [2]:
ds = tfds.load('beans', split=["test", "train", "validation"], shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incomplete6IVY6W/beans-train.tfrecord


  0%|          | 0/1034 [00:00<?, ? examples/s]

...

  0%|          | 0/133 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incomplete6IVY6W/beans-test.tfrecord


  0%|          | 0/128 [00:00<?, ? examples/s]

Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.


In [3]:
ds, info = tfds.load('beans', with_info=True)

print(info)

tfds.core.DatasetInfo(
    name='beans',
    version=0.1.0,
    description='Beans is a dataset of images of beans taken in the field using smartphone
cameras. It consists of 3 classes: 2 disease classes and the healthy class.
Diseases depicted include Angular Leaf Spot and Bean Rust. Data was annotated
by experts from the National Crops Resources Research Institute (NaCRRI) in
Uganda and collected by the Makerere AI research lab.',
    homepage='https://github.com/AI-Lab-Makerere/ibean/',
    features=FeaturesDict({
        'image': Image(shape=(500, 500, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    total_num_examples=1295,
    splits={
        'test': 128,
        'train': 1034,
        'validation': 133,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {beansdata,
        author="Makerere AI Lab",
        title="Bean disease dataset",
        month="January",
        year="2020",
        url="https://git

In [4]:
data_train = tfds.load(name="beans", split="train", batch_size=-1) 
data_valid = tfds.load(name="beans", split="validation", batch_size=-1)
data_test = tfds.load(name="beans", split="test", batch_size=-1)

data_train = tfds.as_numpy(data_train)
data_valid = tfds.as_numpy(data_valid)
data_test = tfds.as_numpy(data_test)

x_train, y_train = data_train["image"], data_train["label"]
x_valid, y_valid = data_valid["image"], data_valid["label"]
x_test, y_test = data_test["image"], data_test["label"]

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [5]:
x_train.shape

(1034, 500, 500, 3)

# LeNet

## Model

In [ ]:
class LeNet(keras.Model):
  def __init__(self, kernel_size=5, kernel_numbers = [6, 16, 120],
               dropout_enable=False, l1_enable=False, l2_enable=False):
    super().__init__()

    self.model = Sequential()

    self.model.add(Input((500, 500, 3,), name="Input"))
    
    conv2D = Conv2D(kernel_numbers[0], kernel_size=kernel_size, strides=4,
                    activation="tanh", name="Conv1", padding="same")

    if l1_enable:
      conv2D.kernel_regularizer = l1()
    elif l2_enable:
      conv2D.kernel_regularizer = l2()
    self.model.add(conv2D)

    if dropout_enable:
      self.model.add(Dropout(rate=0.5, name="Drop1"))
    
    self.model.add(AveragePooling2D(pool_size=3, name="Pool1"))
    
    conv2D = Conv2D(kernel_numbers[1], kernel_size=kernel_size, strides=4,
                    activation="tanh", name="Conv2", padding="same")

    if l1_enable:
      conv2D.kernel_regularizer = l1()
    elif l2_enable:
      conv2D.kernel_regularizer = l2()
    self.model.add(conv2D)
    
    if dropout_enable:
      self.model.add(Dropout(rate=0.5, name="Drop2"))

    self.model.add(AveragePooling2D(pool_size=3, name="Pool2"))
    conv2D = Conv2D(kernel_numbers[2], kernel_size=kernel_size, strides=3,
                    activation="tanh", name="Conv3", padding="same")

    if l1_enable:
      conv2D.kernel_regularizer = l1()
    elif l2_enable:
      conv2D.kernel_regularizer = l2()
    self.model.add(conv2D)
    
    if dropout_enable:
      self.model.add(Dropout(rate=0.5, name="Drop3"))
    
    self.model.add(Flatten(name="Flat"))
    self.model.add(Dense(84, activation="tanh", name="Dense"))
    self.model.add(Dense(3, activation="softmax", name="Output"))

  def call(self, inputs):
    return self.model.call(inputs)

## Regularization Effect

### Without Regularization

In [ ]:
lenet = LeNet(kernel_size=5)
es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
lenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
lenet.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=30, callbacks=[es_callback])

Epoch 1/30
33/33 [==============================] - 14s 404ms/step - loss: 1.0125 - accuracy: 0.4797 - val_loss: 0.8875 - val_accuracy: 0.6015
Epoch 2/30
33/33 [==============================] - 14s 439ms/step - loss: 0.9227 - accuracy: 0.5706 - val_loss: 0.8935 - val_accuracy: 0.5789
Epoch 3/30
33/33 [==============================] - 14s 425ms/step - loss: 0.8902 - accuracy: 0.5986 - val_loss: 0.8651 - val_accuracy: 0.6466
Epoch 4/30
33/33 [==============================] - 14s 420ms/step - loss: 0.8463 - accuracy: 0.6228 - val_loss: 0.8401 - val_accuracy: 0.5940
Epoch 5/30
33/33 [==============================] - 14s 416ms/step - loss: 0.8279 - accuracy: 0.6132 - val_loss: 0.9516 - val_accuracy: 0.5338
Epoch 6/30
33/33 [==============================] - 13s 381ms/step - loss: 0.8493 - accuracy: 0.6025 - val_loss: 0.8659 - val_accuracy: 0.5940
Epoch 7/30
33/33 [==============================] - 13s 388ms/step - loss: 0.8036 - accuracy: 0.6422 - val_loss: 0.8013 - val_accuracy: 0.6090

In [ ]:
lenet.evaluate(x=x_train, y=y_train)
lenet.evaluate(x=x_valid, y=y_valid)
lenet.evaluate(x=x_test, y=y_test)

4/4 [==============================] - 1s 213ms/step - loss: 0.6803 - accuracy: 0.6875


[0.6803126931190491, 0.6875]

### Dropout Regularization

In [ ]:
lenet = LeNet(dropout_enable=True)
es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
lenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
lenet.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=30, callbacks=[es_callback])

Epoch 1/30
33/33 [==============================] - 15s 416ms/step - loss: 1.0481 - accuracy: 0.4429 - val_loss: 0.9382 - val_accuracy: 0.5414
Epoch 2/30
33/33 [==============================] - 14s 410ms/step - loss: 0.9198 - accuracy: 0.5629 - val_loss: 0.8080 - val_accuracy: 0.6617
Epoch 3/30
33/33 [==============================] - 14s 424ms/step - loss: 0.8493 - accuracy: 0.6141 - val_loss: 0.7799 - val_accuracy: 0.6617
Epoch 4/30
33/33 [==============================] - 15s 471ms/step - loss: 0.8576 - accuracy: 0.6151 - val_loss: 0.7649 - val_accuracy: 0.6541
Epoch 5/30
33/33 [==============================] - 15s 446ms/step - loss: 0.8101 - accuracy: 0.6422 - val_loss: 0.8076 - val_accuracy: 0.5940
Epoch 6/30
33/33 [==============================] - 16s 478ms/step - loss: 0.7936 - accuracy: 0.6509 - val_loss: 0.7112 - val_accuracy: 0.7218
Epoch 7/30
33/33 [==============================] - 15s 454ms/step - loss: 0.7518 - accuracy: 0.6683 - val_loss: 0.7306 - val_accuracy: 0.6391

In [ ]:
lenet.evaluate(x=x_train, y=y_train)
lenet.evaluate(x=x_valid, y=y_valid)
lenet.evaluate(x=x_test, y=y_test)

4/4 [==============================] - 1s 217ms/step - loss: 0.5852 - accuracy: 0.7109


[0.5852465033531189, 0.7109375]

### L1 Regularization

In [ ]:
lenet = LeNet(l1_enable=True)
es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
lenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
lenet.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=30, callbacks=[es_callback])

Epoch 1/30
33/33 [==============================] - 13s 387ms/step - loss: 7.2656 - accuracy: 0.4894 - val_loss: 3.2531 - val_accuracy: 0.5564
Epoch 2/30
33/33 [==============================] - 13s 383ms/step - loss: 2.4089 - accuracy: 0.5638 - val_loss: 1.8688 - val_accuracy: 0.5038
Epoch 3/30
33/33 [==============================] - 14s 417ms/step - loss: 1.6831 - accuracy: 0.5861 - val_loss: 1.4678 - val_accuracy: 0.5940
Epoch 4/30
33/33 [==============================] - 13s 383ms/step - loss: 1.3946 - accuracy: 0.6151 - val_loss: 1.2694 - val_accuracy: 0.6617
Epoch 5/30
33/33 [==============================] - 13s 384ms/step - loss: 1.2873 - accuracy: 0.6122 - val_loss: 1.2298 - val_accuracy: 0.6015
Epoch 6/30
33/33 [==============================] - 13s 381ms/step - loss: 1.2283 - accuracy: 0.6315 - val_loss: 1.1712 - val_accuracy: 0.6391
Epoch 7/30
33/33 [==============================] - 13s 382ms/step - loss: 1.2052 - accuracy: 0.6151 - val_loss: 1.1398 - val_accuracy: 0.6391

In [ ]:
lenet.evaluate(x=x_train, y=y_train)
lenet.evaluate(x=x_valid, y=y_valid)
lenet.evaluate(x=x_test, y=y_test)

4/4 [==============================] - 1s 222ms/step - loss: 0.9490 - accuracy: 0.6484


[0.9490329027175903, 0.6484375]

### L2 Regularization

In [ ]:
lenet = LeNet(l2_enable=True)
es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
lenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
lenet.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=30, callbacks=[es_callback])

Epoch 1/30
33/33 [==============================] - 13s 389ms/step - loss: 1.2680 - accuracy: 0.5000 - val_loss: 1.0991 - val_accuracy: 0.5940
Epoch 2/30
33/33 [==============================] - 13s 383ms/step - loss: 1.0111 - accuracy: 0.6431 - val_loss: 0.9281 - val_accuracy: 0.6391
Epoch 3/30
33/33 [==============================] - 13s 382ms/step - loss: 0.9348 - accuracy: 0.6654 - val_loss: 0.8840 - val_accuracy: 0.6466
Epoch 4/30
33/33 [==============================] - 13s 382ms/step - loss: 0.9021 - accuracy: 0.6702 - val_loss: 0.8489 - val_accuracy: 0.6992
Epoch 5/30
33/33 [==============================] - 13s 383ms/step - loss: 0.8768 - accuracy: 0.6789 - val_loss: 0.8689 - val_accuracy: 0.7068
Epoch 6/30
33/33 [==============================] - 13s 383ms/step - loss: 0.8787 - accuracy: 0.6576 - val_loss: 0.8474 - val_accuracy: 0.6391
Epoch 7/30
33/33 [==============================] - 13s 384ms/step - loss: 0.8494 - accuracy: 0.6799 - val_loss: 0.8233 - val_accuracy: 0.6992

In [ ]:
lenet.evaluate(x=x_train, y=y_train)
lenet.evaluate(x=x_valid, y=y_valid)
lenet.evaluate(x=x_test, y=y_test)

4/4 [==============================] - 1s 219ms/step - loss: 0.7910 - accuracy: 0.6797


[0.7909947037696838, 0.6796875]

## Kernel Size Effect

In [ ]:
for kernel_size in [5, 7, 9]:
  print(f"kernel size = {kernel_size}")
  lenet = LeNet(kernel_size=kernel_size)
  es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
  lenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  lenet.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=30, callbacks=[es_callback])

  lenet.evaluate(x=x_train, y=y_train)
  lenet.evaluate(x=x_valid, y=y_valid)
  lenet.evaluate(x=x_test, y=y_test)

kernel size = 5
Epoch 1/30
33/33 [==============================] - 13s 385ms/step - loss: 0.9760 - accuracy: 0.4913 - val_loss: 0.7929 - val_accuracy: 0.6015
Epoch 2/30
33/33 [==============================] - 13s 380ms/step - loss: 0.8182 - accuracy: 0.5996 - val_loss: 0.7611 - val_accuracy: 0.6617
Epoch 3/30
33/33 [==============================] - 12s 379ms/step - loss: 0.7611 - accuracy: 0.6596 - val_loss: 0.7812 - val_accuracy: 0.6992
Epoch 4/30
33/33 [==============================] - 12s 378ms/step - loss: 0.7456 - accuracy: 0.6518 - val_loss: 0.6878 - val_accuracy: 0.7068
Epoch 5/30
33/33 [==============================] - 13s 380ms/step - loss: 0.7325 - accuracy: 0.6702 - val_loss: 0.6722 - val_accuracy: 0.6992
Epoch 6/30
33/33 [==============================] - 12s 378ms/step - loss: 0.7154 - accuracy: 0.6809 - val_loss: 0.6972 - val_accuracy: 0.6917
Epoch 7/30
33/33 [==============================] - 12s 379ms/step - loss: 0.6870 - accuracy: 0.7021 - val_loss: 0.6652 - val_

## Kernel Number Effect

In [ ]:
for kernel_numbers in [
                       [3, 10, 40],
                       [6, 16, 120],
                       [10, 40, 300]]:

  print(f"kernel numbers = {kernel_numbers}")
  lenet = LeNet(kernel_numbers=kernel_numbers)
  es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
  lenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  lenet.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=30, callbacks=[es_callback])

  lenet.evaluate(x=x_train, y=y_train)
  lenet.evaluate(x=x_valid, y=y_valid)
  lenet.evaluate(x=x_test, y=y_test)

kernel numbers = [3, 10, 40]
Epoch 1/30
33/33 [==============================] - 14s 425ms/step - loss: 1.0414 - accuracy: 0.4594 - val_loss: 0.8754 - val_accuracy: 0.5865
Epoch 2/30
33/33 [==============================] - 12s 371ms/step - loss: 0.8532 - accuracy: 0.5948 - val_loss: 0.7496 - val_accuracy: 0.6617
Epoch 3/30
33/33 [==============================] - 12s 371ms/step - loss: 0.7816 - accuracy: 0.6518 - val_loss: 0.7181 - val_accuracy: 0.6692
Epoch 4/30
33/33 [==============================] - 12s 370ms/step - loss: 0.7376 - accuracy: 0.6673 - val_loss: 0.7553 - val_accuracy: 0.6391
Epoch 5/30
33/33 [==============================] - 12s 372ms/step - loss: 0.7968 - accuracy: 0.6364 - val_loss: 0.7713 - val_accuracy: 0.6241
Epoch 6/30
33/33 [==============================] - 12s 373ms/step - loss: 0.7783 - accuracy: 0.6518 - val_loss: 0.7504 - val_accuracy: 0.6090
Epoch 7/30
4/4 [==============================] - 1s 210ms/step - loss: 0.7145 - accuracy: 0.6953
kernel numbers 

## Best Setting

In [ ]:
lenet = LeNet(kernel_size=5, kernel_numbers=[10, 40, 300], dropout_enable=True)
es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
lenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
lenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

lenet.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=30, callbacks=[es_callback])

lenet.evaluate(x=x_train, y=y_train)
lenet.evaluate(x=x_valid, y=y_valid)
lenet.evaluate(x=x_test, y=y_test)

Epoch 1/30
33/33 [==============================] - 14s 399ms/step - loss: 0.9390 - accuracy: 0.5532 - val_loss: 0.7473 - val_accuracy: 0.7068
Epoch 2/30
33/33 [==============================] - 13s 385ms/step - loss: 0.7935 - accuracy: 0.6499 - val_loss: 0.7326 - val_accuracy: 0.6617
Epoch 3/30
33/33 [==============================] - 13s 387ms/step - loss: 0.7805 - accuracy: 0.6489 - val_loss: 0.6819 - val_accuracy: 0.6767
Epoch 4/30
33/33 [==============================] - 13s 385ms/step - loss: 0.7824 - accuracy: 0.6480 - val_loss: 0.6740 - val_accuracy: 0.6992
Epoch 5/30
33/33 [==============================] - 13s 391ms/step - loss: 0.8019 - accuracy: 0.6267 - val_loss: 0.6795 - val_accuracy: 0.6992
Epoch 6/30
33/33 [==============================] - 13s 385ms/step - loss: 0.7306 - accuracy: 0.6712 - val_loss: 0.6887 - val_accuracy: 0.6842
Epoch 7/30
33/33 [==============================] - 13s 384ms/step - loss: 0.7360 - accuracy: 0.6750 - val_loss: 0.6622 - val_accuracy: 0.6917

[0.5686240792274475, 0.7578125]

# Inception

## Model

In [6]:
class Inception(keras.Model):
  def __init__(self):
    super().__init__()
    self.base_model = InceptionV3(
        input_shape=(500, 500, 3), include_top=False, weights='imagenet')

    inputs = Input((500, 500, 3,))
    dropout_layer = Dropout(rate=0.5)
    pooling_layer = GlobalMaxPooling2D(data_format="channels_last")
    output_layer = Dense(3, activation="softmax")

    x = self.base_model(inputs)
    x = pooling_layer(x)
    x = dropout_layer(x)
    outputs = output_layer(x)

    self.model = tf.keras.Model(inputs, outputs)

    self.freeze_base_model()

  def freeze_base_model(self):
    self.base_model.trainable = False

  def unfreeze_partial_base_model(self, number_freeze_layers):
    self.base_model.trainable = True

    for layer in self.base_model.layers[:number_freeze_layers]:
      layer.trainable = False
    
  def call(self, inputs):
    return self.model.call(inputs)

## Results

In [15]:
len(inception.base_model.layers)

311

In [17]:
  for number_freeze_layers in [0, 50, 100, 200, 311]:
  print(f"number freeze layers = {number_freeze_layers}")
  
  inception = Inception()

  es_callback = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
  inception.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  history = inception.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), epochs=30, callbacks=[es_callback])

  inception.evaluate(x=x_train, y=y_train)
  inception.evaluate(x=x_valid, y=y_valid)
  inception.evaluate(x=x_test, y=y_test)

  if number_freeze_layers < 311:
    inception.unfreeze_partial_base_model(number_freeze_layers=number_freeze_layers)

    fine_tuning_history = inception.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid),
                                    epochs=30, initial_epoch=history.epoch[-1]+1, callbacks=[es_callback])
    
    inception.evaluate(x=x_train, y=y_train)
    inception.evaluate(x=x_valid, y=y_valid)
    inception.evaluate(x=x_test, y=y_test)

  print("============================\n")

number freeze layers = 0
Epoch 1/30
33/33 [==============================] - 36s 925ms/step - loss: 74.2033 - accuracy: 0.3288 - val_loss: 30.9558 - val_accuracy: 0.3910
Epoch 2/30
33/33 [==============================] - 29s 870ms/step - loss: 55.9643 - accuracy: 0.3549 - val_loss: 21.7825 - val_accuracy: 0.4135
Epoch 3/30
33/33 [==============================] - 28s 862ms/step - loss: 53.8955 - accuracy: 0.3888 - val_loss: 32.3487 - val_accuracy: 0.5038
Epoch 4/30
33/33 [==============================] - 29s 869ms/step - loss: 42.9981 - accuracy: 0.4178 - val_loss: 20.8096 - val_accuracy: 0.5038
Epoch 5/30
33/33 [==============================] - 28s 863ms/step - loss: 37.8272 - accuracy: 0.4439 - val_loss: 21.3377 - val_accuracy: 0.5038
Epoch 6/30
33/33 [==============================] - 29s 868ms/step - loss: 34.6697 - accuracy: 0.4584 - val_loss: 14.7510 - val_accuracy: 0.4737
Epoch 7/30
33/33 [==============================] - 28s 862ms/step - loss: 30.6389 - accuracy: 0.4536 - v